# Projeto 2 - Ciência dos Dados

**Nome:** João Pedro Henneberg Ribeiro

**Nome:** João Nogueira Roxo da Fonseca

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@JoaoHenneberg***

In [3]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [4]:
#Produto escolhido:
produto = 'Gabigol'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 500

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [5]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [6]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [7]:
dados_raw = pd.read_excel("Gabigol.xlsx")

In [8]:
dados = set(dados_raw.Treinamento)

In [9]:
dados_ = pd.Series(list(dados))

In [10]:
dados_.to_excel("Gabigol_base.xlsx")

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [57]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [243]:
#Importando as Tabelas
tabela_teste = pd.read_excel('Gabigol.xlsx', sheet_name = "Teste")
tabela_treinamento =pd.read_excel('Gabigol.xlsx', sheet_name = "Treinamento")

In [251]:
#Classificando os Tweets
ruim = tabela_treinamento.loc[tabela_treinamento["Classificação"] == 0]
bom = tabela_treinamento.loc[tabela_treinamento["Classificação"] == 1]
indiferente = tabela_treinamento.loc[tabela_treinamento["Classificação"] == 2]

In [60]:
#Colocando a Tabela em um Texto
ruim_texto = " ".join(ruim["Treinamento"])
bom_texto = " ".join(bom["Treinamento"])
indiferente_texto = " ".join(indiferente["Treinamento"])

In [61]:
#Limpando o Código(Limpando links)
ruim_texto = re.sub(r"http\S+", "", ruim_texto)
bom_texto = re.sub(r"http\S+", "", bom_texto)
indiferente_texto = re.sub(r"http\S+", "", indiferente_texto)

In [62]:
#Limpando o Código(Limpando o resto)
ruim_limpo = cleanup(ruim_texto.lower())
bom_limpo = cleanup(bom_texto.lower())
indiferente_limpo = cleanup(indiferente_texto.lower())

In [63]:
#Criando a Tabela de Frequências
freq_ruim = pd.Series(ruim_limpo.split())
freq_bom = pd.Series(bom_limpo.split())
freq_indiferente = pd.Series(indiferente_limpo.split())

In [64]:
#Juntando as Palavras Repetidas
tabela_ruim = freq_ruim.value_counts()
tabela_bom = freq_bom.value_counts()
tabela_indiferente = freq_indiferente.value_counts()

In [65]:
#Criando a Tabela com a Frequência Relativa
tabela_ruim_relativa = freq_ruim.value_counts(True)
tabela_bom_relativa = freq_bom.value_counts(True)
tabela_indiferente_relativa = freq_indiferente.value_counts(True)

In [66]:
#Definindo a União Entre as Tabelas de Bom/Indiferente e Ruim/Indiferente
set_ruim = set(tabela_ruim_relativa.index)
set_bom = set(tabela_bom_relativa.index)
set_indiferente = set(tabela_indiferente_relativa.index)

uniao_ruim_ind = set_ruim.union(set_indiferente)
uniao_bom_ind = set_bom.union(set_indiferente)

uniao_total = uniao_ruim_ind.union(uniao_bom_ind)

In [67]:
#Definindo o Total para ser Usado no Cálculo da Probabilidade
ruim_absoluto = tabela_ruim + 1
total_ruim = sum(ruim_absoluto) + len(uniao_total)

bom_absoluto = tabela_bom + 1
total_bom = sum(bom_absoluto) + len(uniao_total)

indiferente_absoluto = tabela_indiferente + 1
total_indiferente = sum(indiferente_absoluto) + len(uniao_total)

In [147]:
#Calculando a Palavra Dado Bom/Ruim/Indiferente
prob_ruim = tabela_ruim.sum()/total_ruim

prob_bom = tabela_bom.sum()/total_bom

prob_indiferente = tabela_indiferente.sum()/total_indiferente

In [123]:
#Probabilidade do Tweet ser Ruim
prob_ruim

0.28613763582482715

In [124]:
#Probabilidade do Tweet ser Bom
prob_bom

0.4185053380782918

In [125]:
#Probabilidade do Tweet ser Indiferente
prob_indiferente

0.5522745411013568

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [127]:
#Montando Fórmula de Classificação dos Tweets
def concatena(tweet):
    return tweet

tweets_juntos = tabela_teste.Teste.apply(concatena)

In [128]:
tabela_teste = pd.read_excel('Gabigol.xlsx', sheet_name = "Teste")

In [159]:
tweets_varridos = []
for mensagem in tweets_juntos:
    tweets_varridos.append(cleanup(mensagem.lower())) 

In [184]:
#Criando o Código de Classificação
lista_classificacao = []

d_lista_bom = tabela_bom[1]
d_lista_ruim = tabela_ruim[1]
d_lista_indiferente = tabela_indiferente[1]
d = d_lista_bom + d_lista_ruim + d_lista_indiferente

def probabilidade(palavra):
    if palavra in tabela_bom:
        k = tabela_bom[palavra]
    else:
        k = 0
    if palavra in tabela_ruim:
        J = tabela_ruim[palavra]
    else:
        J = 0
    if palavra in tabela_indiferente:
        l = tabela_indiferente[palavra]
    else:
        l = 0
    prob_bom =( k +1)/(len(freq_bom) + d_lista_bom)
    prob_ruim =(J + 1)/(len(freq_ruim) + d_lista_ruim) 
    prob_indiferente =(l + 1)/(len(freq_indiferente) + d_lista_indiferente) 
    
    return [prob_bom, prob_ruim, prob_indiferente]


for tweet in tweets_varridos:
    periodo = tweet.split()
    lista_palavras = []
    for elemento in periodo:
        lista_palavras.append(elemento)
        bom = 1
        ruim = 1
        indiferente = 1
        for palavra in lista_palavras:
            if palavra in tabela_bom_relativa and palavra in tabela_indiferente_relativa and palavra in tabela_ruim_relativa:
                bom *= tabela_bom_relativa[palavra]                                                           #Normal
                ruim *= tabela_ruim_relativa[palavra]                                                         #Normal
                indiferente *= tabela_indiferente_relativa[palavra]                                           #Normal
            elif palavra in tabela_bom_relativa and palavra not in tabela_indiferente_relativa and palavra not in tabela_ruim_relativa:
                bom *= tabela_bom_relativa[palavra]                                                           #Normal
                ruim = probabilidade(palavra)[1]*ruim                                                         #LaPlace
                indiferente = probabilidade(palavra)[2]*indiferente                                           #LaPlace    
            elif palavra in tabela_bom_relativa and palavra not in tabela_indiferente_relativa and palavra in tabela_ruim_relativa:
                bom *= tabela_bom_relativa[palavra]                                                           #Normal
                ruim *= tabela_ruim_relativa[palavra]                                                         #Normal
                indiferente = probabilidade(palavra)[2]*indiferente                                           #LaPlace    
            elif palavra in tabela_bom_relativa and palavra in tabela_indiferente_relativa and palavra not in tabela_ruim_relativa:
                bom *= tabela_bom_relativa[palavra]                                                           #Normal
                ruim = probabilidade(palavra)[1]*ruim                                                         #LaPlace
                indiferente *= tabela_indiferente_relativa[palavra]                                           #Normal    
            
            elif palavra not in tabela_bom_relativa and palavra in tabela_indiferente_relativa and palavra in tabela_ruim_relativa:
                bom = probabilidade(palavra)[0]*bom                                                           #laPlace
                ruim *= tabela_ruim_relativa[palavra]                                                         #Normal
                indiferente *= tabela_indiferente_relativa[palavra]                                           #Normal
            elif palavra not in tabela_bom_relativa and palavra not in tabela_indiferente_relativa and palavra in tabela_ruim_relativa:
                bom = probabilidade(palavra)[0]*bom                                                           #laPlace
                ruim = probabilidade(palavra)[1]*ruim                                                         #Normal
                indiferente = probabilidade(palavra)[2]*indiferente                                           #LaPlace  
            elif palavra not in tabela_bom_relativa and palavra in tabela_indiferente_relativa and palavra not in tabela_ruim_relativa:
                bom = probabilidade(palavra)[0]*bom                                                           #laPlace
                ruim = probabilidade(palavra)[1]*ruim                                                         #LaPlace
                indiferente *= tabela_indiferente_relativa[palavra]                                           #Normal
            else:
                bom = probabilidade(palavra)[0]*bom                                                           #laPlace
                ruim = probabilidade(palavra)[1]*ruim                                                         #LaPlace
                indiferente = probabilidade(palavra)[2]*indiferente                                           #LaPlace    
            
        prob_gosta_frase = prob_bom * bom 
        prob_desgosta_frase = prob_ruim * ruim
        prob_indiferente_frase = prob_indiferente * indiferente
        
        if prob_gosta_frase > prob_desgosta_frase and prob_gosta_frase > prob_indiferente_frase:
            lista_classificacao.append("1")
        if prob_desgosta_frase > prob_gosta_frase and prob_desgosta_frase > prob_indiferente_frase:
            lista_classificacao.append("0")
        else:
            lista_classificacao.append("2")
            

In [232]:
#Calculando a Porcentagem de Bom/Ruim/Indiferente do Classificador
pos = 0
falso_pos = 0
neg = 0
falso_neg = 0
total = len(tabela_teste["Classificação"])
irrel = 0
bom = 0
ruim = 0

for [e,i] in zip(tabela_teste["Classificação"], tabela_teste["Teste"]):
    if e == 0:
        if i == 0:
            pos += 1
        else: 
            falso_neg += 1
    elif e == 1:
        if i == 1:
            pos += 1
        else: 
            falso_neg += 1
    elif e == 2:            
        if i == 2:
            neg +=1
        else:
            falso_pos +=1        
            
for e in tabela_teste["Classificação"]:
    if e==1:
        bom +=1
    elif e==2:
        irrel +=1
    else:
        ruim += 1
        
porcentagem_bom = bom*100/300
porcentagem_ruim = ruim*100/300
porcentagem_irrel = irrel*100/300

In [233]:
porcentagem_bom

34.666666666666664

In [234]:
porcentagem_ruim

8.0

In [235]:
porcentagem_irrel

57.333333333333336

In [257]:
#Calculando a Porcentagem da Classificação Feita Manualmente
porcentagem_bom_manual = (len(bom)/450)*100
porcentagem_ruim_manual = (len(ruim)/450)*100
porcentagem_irrel_manual = (len(indiferente)/450)*100

In [258]:
porcentagem_bom_manual

26.222222222222225

In [259]:
porcentagem_ruim_manual

13.111111111111112

In [260]:
porcentagem_irrel_manual

60.66666666666667

### Concluindo

Com a análise das porcentagens de acerto do classificador, podemos concluir que apesar de ter análisado corretamente grande parte dos tweets, o classificador do código apresenta certas falhas(8,4% em Tweets "Bons", 5,1% em Tweets "Ruins" e 3,3% em Tweets "Indiferentes"). Isso pode ocorrer devido a palavras nos tweets usam um certo tipo de ironia, ou até dupla negação. Como a ironia se classifica como uma inversão do real significado de algo em determinado contexto, é comum o código não interpretar tais ironias, já  que somente são compreendidas com a compreensão do contexto em que a palavra está inserida.

Uma possível melhoria para o código poderia ser implementar mais parâmetros para divisão das categorias, ao invés de somente 3 como feito no código. Com isso, seria possível fazer uma varredura mais precisa no código, já que teriam mais possibilidades de classificação de diversos tweets diferentes.

Além do aumento das divisões dos códigos, outra possível saída para a redução dos erros seria de um classificador com um código mais aprofundado, que baseado nas palavras no contexto do Tweet, seja capaz de identificar se uma palavra específica está sendo usada no seu sentido conotativo ou denotativo. Com tal aprofundamento, as classificações seriam muito mais precisas e exatas.

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**

https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/ **Link que ajuda nas probabilidades**